# Zep Vector Store
## A long-term memory store for LLM applications
This notebook demonstrates how to use the Zep Vector Store with LlamaIndex.


## About Zep
Zep makes it easy for developers to add relevant documents, chat history memory & rich user data to their LLM app's prompts. 


## Note
Zep can automatically embed your documents. The LlamaIndex implementation of the Zep Vector Store utilizes LlamaIndex's embedders to do so.


## Getting Started
**Quick Start Guide:** https://docs.getzep.com/deployment/quickstart/
**GitHub:** https://github.com/getzep/zep

In [1]:
import logging
import sys
from uuid import uuid4

from dotenv import load_dotenv

load_dotenv()

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ZepVectorStore

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [3]:
# load documents
documents = SimpleDirectoryReader("../data/paul_graham").load_data()

# Create a Zep Vector Store and Index

You can use an existing Zep Collection, or create a new one.

In [4]:
from llama_index.storage.storage_context import StorageContext

zep_api_url = "http://localhost:8000"
collection_name = f"graham{uuid4().hex}"
file = "babbages_calculating_engine.txt"

vector_store = ZepVectorStore(
    api_url=zep_api_url, collection_name=collection_name, embedding_dimensions=1536
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/collection/grahamaed555f7b2354d0bb981b62d2e90741e "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8000/api/v1/collection/grahamaed555f7b2354d0bb981b62d2e90741e "HTTP/1.1 404 Not Found"
INFO:llama_index.vector_stores.zep:Collection grahamaed555f7b2354d0bb981b62d2e90741e does not exist, will try creating one with dimensions=1536
Collection grahamaed555f7b2354d0bb981b62d2e90741e does not exist, will try creating one with dimensions=1536
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collection/grahamaed555f7b2354d0bb981b62d2e90741e "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/api/v1/collection/grahamaed555f7b2354d0bb981b62d2e90741e "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/collection/grahamaed555f7b2354d0bb981b62d2e90741

In [5]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")

print(str(response))

INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collection/grahamaed555f7b2354d0bb981b62d2e90741e/search?limit=2 "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/api/v1/collection/grahamaed555f7b2354d0bb981b62d2e90741e/search?limit=2 "HTTP/1.1 200 OK"

The author grew up writing short stories and programming on an IBM 1401. He also convinced his father to buy him a TRS-80 microcomputer, which he used to write simple games, a program to predict how high his model rockets would fly, and a word processor. He then went to college to study philosophy, but switched to AI after becoming bored with the courses.


In [6]:
from llama_index.schema import TextNode

nodes = [
    TextNode(
        text="The Shawshank Redemption",
        metadata={
            "author": "Stephen King",
            "theme": "Friendship",
        },
    ),
    TextNode(
        text="The Godfather",
        metadata={
            "director": "Francis Ford Coppola",
            "theme": "Mafia",
        },
    ),
    TextNode(
        text="Inception",
        metadata={
            "director": "Christopher Nolan",
        },
    ),
]

In [7]:
collection_name = f"movies{uuid4().hex}"

vector_store = ZepVectorStore(
    api_url=zep_api_url, collection_name=collection_name, embedding_dimensions=1536
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex(nodes, storage_context=storage_context)

INFO:httpx:HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8000/healthz "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/collection/moviesfbb23b29db4b405c8adf9891cd42a2ae "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8000/api/v1/collection/moviesfbb23b29db4b405c8adf9891cd42a2ae "HTTP/1.1 404 Not Found"
INFO:llama_index.vector_stores.zep:Collection moviesfbb23b29db4b405c8adf9891cd42a2ae does not exist, will try creating one with dimensions=1536
Collection moviesfbb23b29db4b405c8adf9891cd42a2ae does not exist, will try creating one with dimensions=1536
INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collection/moviesfbb23b29db4b405c8adf9891cd42a2ae "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/api/v1/collection/moviesfbb23b29db4b405c8adf9891cd42a2ae "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8000/api/v1/collection/moviesfbb23b29db4b405c8adf9891cd42a2a

In [8]:
from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

filters = MetadataFilters(filters=[ExactMatchFilter(key="theme", value="Mafia")])

In [9]:
retriever = index.as_retriever(filters=filters)
result = retriever.retrieve("What is inception about?")

for r in result:
    print("\n", r.node.text, r.score)

INFO:httpx:HTTP Request: POST http://localhost:8000/api/v1/collection/moviesfbb23b29db4b405c8adf9891cd42a2ae/search?limit=2 "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8000/api/v1/collection/moviesfbb23b29db4b405c8adf9891cd42a2ae/search?limit=2 "HTTP/1.1 200 OK"

 The Godfather 0.884073254953458
